<a href="https://colab.research.google.com/github/ibrahimuk/pandas/blob/master/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Travaux pratiques : Développement d'un moteur de recherche avec TF-Idf
Dans ce TP, nous allons développer un moteur de recherche basé sur la technique TF-Idf (Term Frequency-Inverse Document Frequency) appliquée aux critiques de films issues de la base de données IMDB. L'objectif principal de ce TP est d'apprendre à prétraiter des données textuelles, à transformer des textes en représentations numériques exploitables et à construire un système de recherche permettant de retrouver des documents pertinents en fonction de requêtes utilisateur.

## Objectifs
* Prétraitement des Données Textuelles : Apprendre à nettoyer et préparer les données textuelles en éliminant les balises HTML, en convertissant les textes en minuscules, et en appliquant la lemmatisation pour normaliser les mots.
* Transformation en Matrice TF-Idf : Utiliser le vectoriseur TF-Idf pour convertir les critiques de films en une matrice numérique où chaque dimension représente l'importance d'un terme dans un document.
*Développement d'un moteur de recherche : Implémenter un moteur de recherche simple qui utilise la similarité cosinus pour retrouver les critiques les plus pertinentes par rapport à une requête utilisateur.

## Outils Utilisés
* Pandas : Pour le chargement et la manipulation des données.
* NLTK (Natural Language Toolkit) : Pour les opérations de prétraitement telles que la lemmatisation et l'élimination des stop words.
* Scikit-learn : Pour la transformation des textes en matrices TF-Idf et l'implémentation des algorithmes de recherche.

## Étapes du TP
* Chargement des données : Importer les critiques de films à partir d'un fichier CSV et afficher les premières lignes pour comprendre la structure des données.
* Nettoyage et prétraitement : Appliquer des techniques de nettoyage des textes, comme la suppression des balises HTML et la conversion en minuscules. Utiliser NLTK pour lemmatiser les mots, ce qui permet de réduire les variations grammaticales.
* Transformation TF-Idf : Initialiser un TfidfVectorizer pour convertir les critiques en une matrice TF-Idf. Cette matrice permettra de quantifier l'importance des termes dans chaque critique.
* Développement du moteur de recherche : Implémenter une fonction de recherche qui utilise la similarité cosinus pour mesurer la pertinence des critiques par rapport à une requête donnée. Afficher les résultats de recherche de manière lisible.
* Évaluation et discussion : Tester le moteur de recherche avec différentes requêtes, analyser les résultats obtenus, et discuter des améliorations possibles pour optimiser les performances du système.

In [1]:
# from google.colab import files
# uploaded = files.upload()

# Importation de la base de donnée à partir local
# import io
# df = pd.read_csv(io.BytesIO(uploaded['IMDB.csv']))
# Dataset is now stored in a Pandas Dataframe

## Chargement des données

In [2]:
import pandas as pd

# Charger le dataset
df = pd.read_csv('IMDB.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'IMDB.csv'

In [ ]:
# Imprimer la première ligne du dataframe
print(df['review'].loc[0])

In [ ]:
print(df.head())

## Prétraitement des données

Cette section du code se concentre sur le prétraitement des critiques de films afin de les préparer pour une analyse ultérieure. Le prétraitement est une étape cruciale dans le traitement du langage naturel (NLP) car il permet de nettoyer et de normaliser les données textuelles.

1. **Importation des Bibliothèques NLTK :**

* `nltk` est importé avec des modules spécifiques tels que stopwords et wordnet pour le traitement linguistique.
* `WordNetLemmatizer` est importé pour effectuer la lemmatisation, une technique qui réduit les mots à leur forme de base.

2. **Nettoyage des Données :**

Les critiques de films, stockées dans la colonne review du dataframe df, sont nettoyées en supprimant les balises HTML (`<br />`) et en convertissant tout le texte en minuscules. Cela permet d'uniformiser les données et de faciliter l'analyse.

3. **Définition de la Fonction de Lemmatisation :**

La fonction `lemmatize_text` prend un texte en entrée, le divise en mots (`split`), et lemmatise chaque mot en utilisant `WordNetLemmatizer`, en considérant chaque mot comme un verbe (`pos=wordnet.VERB`), et en excluant les stop words.

4. **Application de la Lemmatisation :**

La fonction `lemmatize_text` est appliquée à chaque critique de film dans la colonne `review` du dataframe `df`.

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import numpy as np
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import re

# Nettoyer les données : suppression des balises HTML et conversion en minuscules
df['review'] = df['review'].str.replace('<br />', '')  # Suppression des balises HTML
df['review'] = df['review'].str.lower()  # Conversion en minuscules des données

# Initialiser le lemmatiseur WordNet
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))  # Chargement des stop words en anglais

# Fonction de lemmatisation
def lemmatize_text(text):
  ############## Code ############

    # Create a lemmatizer object
    lemmatizer = WordNetLemmatizer()

    # Tokenize the text into words
    words = text.split()

    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Join the lemmatized words into a lemmatized text string
    ################################
    return ' '.join(lemmatized_words)

# Appliquer la lemmatisation
df['review'] = df['review'].apply(lemmatize_text)

# Afficher la première ligne du dataset prétraité
print(df['review'].loc[0])

## Transformation en matrice TF-IDF

Cette section du code concerne la transformation des critiques de films prétraitées en une matrice TF-IDF (Term Frequency-Inverse Document Frequency).

1. **Importation du Module TfidfVectorizer de scikit-learn :**

Le `TfidfVectorizer` de la bibliothèque `scikit-learn` est utilisé pour convertir les textes en une matrice TF-IDF. Cette méthode permet de quantifier l'importance des termes dans les documents en tenant compte de leur fréquence dans les documents et de leur rareté dans l'ensemble des documents.

2. **Initialisation du Vectorizer :**

Un objet `TfidfVectorizer` est initialisé avec les paramètres suivants :
* `max_features=1000` : Limite le nombre de termes à 1000, en se concentrant sur les termes les plus fréquents.
* `stop_words='english'` : Exclut les stop words anglais, qui sont des mots courants n'apportant pas beaucoup de sens (comme "the", "is", etc.).

3. **Transformation des données :**

Le `fit_transform` est appliqué sur les critiques de films prétraitées stockées dans `df['review']`. Cette méthode effectue typiquement deux opérations :
* `fit` : Apprend le vocabulaire du texte et calcule les statistiques TF-IDF pour chaque terme.
* `transform` : Convertit les critiques en vecteurs TF-IDF en utilisant les statistiques apprises.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english' )

X = vectorizer.fit_transform(df['review'])

vectorizer.get_feature_names_out()

print(X.shape)

(4, 9)

# Initialiser le vectorizer
# def lemmatize_text(text):
  ############## Code ############


  ################################

# Ajuster sur les données et transformer les données
# def lemmatize_text(text):
  ############## Code ############


  ################################
print("Shape of TF-IDF matrix:", X.shape)


In [ ]:
X[0].toarray()

## Implémentation du moteur de Recherche
Cette section du code implémente un moteur de recherche qui permet de retrouver les critiques de films les plus pertinentes en fonction d'une requête utilisateur. Le moteur de recherche utilise la similarité cosinus pour mesurer la pertinence des critiques par rapport à la requête.

1. **Importation des bibliothèques nécessaires :**

* `numpy` pour la manipulation des tableaux et le tri des indices.
* `cosine_similarity` de `sklearn.metrics.pairwise` pour calculer la similarité entre les vecteurs TF-IDF.

2. **Définition de la fonction search :**

La fonction `search` prend une requête utilisateur, le vectoriseur TF-IDF, la matrice TF-IDF des critiques de films, le dataframe contenant les critiques, et le nombre de résultats à retourner (`top_n`).

3. **Transformation de la Requête en Vecteur TF-IDF :**

La requête utilisateur est transformée en vecteur TF-IDF en utilisant le vectoriseur initialisé précédemment.

4. **Calcul de la similarité cosinus :**

La similarité cosinus est calculée entre le vecteur TF-IDF de la requête et la matrice TF-IDF des critiques. La similarité cosinus mesure la proximité entre deux vecteurs dans un espace multidimensionnel.

5. **Tri des critiques par similarité :**

* Les indices des critiques sont triés en ordre décroissant de similarité.
* Les indices des `top_n` critiques les plus similaires sont sélectionnés.
* La fonction retourne les indices des critiques les plus similaires et leurs scores de similarité.


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def search(query, vectorizer, tfidf_matrix, df, top_n=5):
    # Transformer la requête en vecteur TF-Idf
    query_vec = vectorizer.transform([query])

    # Calculer la similarité cosinus entre la requête et les critiques
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # Trouver les indices des critiques les plus similaires
    indices = np.argsort(similarities)[::-1][:top_n]

    # Retourner les critiques les plus similaires
    return indices, similarities[indices]

# Exemple d'utilisation
query = "great movie with excellent acting"
indices, similarities = search(query, vectorizer, X, df)

# Afficher les résultats
for idx, similarity in zip(indices, similarities):
    print(f"Similarity: {similarity:.4f}")
    print(f"Review: {df.iloc[idx]['review']}\n")
    print(f"Sentiment: {df.iloc[idx]['sentiment']}\n")
    print("="*50 + "\n")